In [1]:
import numpy as np
import fermion as f
import matplotlib.pyplot as plt
from fermion import Math as fm
from pfapack import pfaffian as pfa
import scipy as sp

from timeit import default_timer as timer

In [2]:
n=30

In [3]:
dq = f.Operator.double_quantum(n)
Z = f.Operator.global_Z(n)
time_evo = f.Unitary(dq,.05, 2.05)

In [4]:
center = int(n/2)
pauli_pairs = [["X","X"], ["Y","Y"], ["Z","Z"]]

In [5]:
def d_weight(pauli):
    weight = 1
    if pauli[0] != "Z":
        weight *= -.5
    if pauli[2] != "Z":
        weight *= -.5
    return weight

In [6]:
start = timer()
dip_corr = np.zeros(len(time_evo.t))
for a in range(n):
    for b in range(a):
        for c in range(n):
            for d in range(c):
                val = np.zeros(len(time_evo.t),dtype=np.complex128)
                for pair1 in pauli_pairs:
                    for pair2 in pauli_pairs:
                        term = pair1+pair2
                        pstring = f.PauliString(term, [a,b,c,d], [True,True,False,False])
                        val = val +  (d_weight(term) * time_evo.pauli_string_expectation(pstring))
                dip_corr = dip_corr + (2/(abs(a-b)**3)) * (2/(abs(c-d)**3)) * val
end = timer()
print(end-start)

C:\Users\awsta\Documents\workspace\qeg-env\lib\site-packages\pfapack\pfaffian.py:301: RuntimeWarning: invalid value encountered in true_divide
  tau = tau / A[k, k + 1]
C:\Users\awsta\Documents\workspace\qeg-env\lib\site-packages\pfapack\pfaffian.py:301: RuntimeWarning: divide by zero encountered in true_divide
  tau = tau / A[k, k + 1]
C:\Users\awsta\Documents\workspace\qeg-env\lib\site-packages\numpy\core\numeric.py:936: RuntimeWarning: invalid value encountered in multiply
  return multiply(a.ravel()[:, newaxis], b.ravel()[newaxis, :], out)
C:\Users\awsta\Documents\workspace\qeg-env\lib\site-packages\pfapack\pfaffian.py:310: RuntimeWarning: invalid value encountered in subtract
  A[k + 2 :, k + 2 :] = A[k + 2 :, k + 2 :] - np.outer(


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots()
ax.set_title('<Dz(t)Dz>, H=DQ, N=30')
c = ax.plot(time_evo.t, [sig/n for sig in dip_corr],'b', time_evo.t, [0 for _ in dip_corr], 'g')
ax.legend(['<Dz(t)Dz>', '0'])
ax.set_xlabel('J t')
plt.savefig("DzDz_tpc.png")
plt.show()

In [ ]:
import pickle

In [ ]:
with open('dzdz_tpc.dat', 'wb') as f:
    pickle.dump(dip_corr, f)